***GENERATED CODE FOR thoseatriskofonlinesexwork PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
from pyspark.ml.feature import StringIndexer
import json
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import round
from pyspark.ml.feature import Binarizer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def BinarizerTransform(df, params, transformationData={}):
    dfReturn = df
    transform_params = params
    feature = transform_params['feature']
    outcol = feature + "_binarizer"
    dfReturn = dfReturn.withColumn("feature_cast", dfReturn[feature].cast("double")).drop(feature)\
        .withColumnRenamed("feature_cast", feature)

    dfReturn = dfReturn.fillna({feature: 0.0})
    binarizer = Binarizer(threshold=float(
        transformationData['threshold']), inputCol=feature, outputCol=outcol)
    binarizedDataFrame = binarizer.transform(dfReturn)

    # binarizedDataFrame=binarizedDataFrame.drop(feature).withColumnRenamed(outcol,feature)

    dfReturn = binarizedDataFrame
    dfReturn = dfReturn.withColumn(feature, round(dfReturn[feature], 2))

    return dfReturn


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = BinarizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'User_ID', 'threshold': 12441.148, 'transformation_label': 'Binarizer'}], 'feature': 'User_ID', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
                                              'count': '500', 'mean': '12441.15', 'stddev': '8613.12', 'min': '1.0', 'max': '48294.0', 'missing': '0'}, 'transformation': [{'transformation': 'Binarizer', 'selectedAsDefault': 1}], 'updatedLabel': 'User_ID'}, {'feature_label': 'User_ID', 'threshold': 12441.148, 'transformation_label': 'Binarizer'})
        transformationDF = transformationDF.drop('User_ID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'female', 'max': 'male', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'feature_label': 'Gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Gender')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Age', 'transformation_label': 'String Indexer'}], 'feature': 'Age', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '38.4', 'stddev': '10.68', 'min': '20,7', 'max': '86,6', 'missing': '0', 'distinct': '247'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Age'}, {'feature_label': 'Age', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Age')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Location', 'transformation_label': 'String Indexer'}], 'feature': 'Location', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'A', 'max': 'S', 'missing': '0', 'distinct': '15'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Location'}, {'feature_label': 'Location', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Location')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Verification', 'transformation_label': 'String Indexer'}], 'feature': 'Verification', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Administrator', 'max': 'Verified', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Verification'}, {'feature_label': 'Verification', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Verification')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Sexual_orientation', 'transformation_label': 'String Indexer'}], 'feature': 'Sexual_orientation', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '499', 'mean': '', 'stddev': '', 'min': 'Heterosexual', 'max': 'bisexual', 'missing': '1', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Sexual_orientation'}, {'feature_label': 'Sexual_orientation', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Sexual_orientation')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Sexual_polarity', 'transformation_label': 'String Indexer'}], 'feature': 'Sexual_polarity', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '499', 'mean': '', 'stddev': '', 'min': 'Dominant', 'max': 'Switch', 'missing': '1', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Sexual_polarity'}, {'feature_label': 'Sexual_polarity', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Sexual_polarity')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Looking_for', 'transformation_label': 'String Indexer'}], 'feature': 'Looking_for', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '495', 'mean': '', 'stddev': '', 'min': 'Men', 'max': 'Women', 'missing': '5', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Looking_for'}, {'feature_label': 'Looking_for', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Looking_for')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Points_Rank', 'transformation_label': 'String Indexer'}], 'feature': 'Points_Rank', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '27.84', 'stddev': '83.83', 'min': '-100', 'max': 'a', 'missing': '0', 'distinct': '83'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Points_Rank'}, {'feature_label': 'Points_Rank', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Points_Rank')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Last_login', 'transformation_label': 'String Indexer'}], 'feature': 'Last_login', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'before_1003_days', 'max': 'before_9_days', 'missing': '0', 'distinct': '250'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Last_login'}, {'feature_label': 'Last_login', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Last_login')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Member_since', 'transformation_label': 'String Indexer'}], 'feature': 'Member_since', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '1.11.2009', 'max': '9.9.2015', 'missing': '0', 'distinct': '112'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Member_since'}, {'feature_label': 'Member_since', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Member_since')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Number_of_Comments_in_public_forum', 'transformation_label': 'String Indexer'}], 'feature': 'Number_of_Comments_in_public_forum', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '20.59', 'stddev': '93.3', 'min': '0', 'max': '96', 'missing': '0', 'distinct': '65'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Number_of_Comments_in_pub...'}, {'feature_label': 'Number_of_Comments_in_public_forum', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'Number_of_Comments_in_public_forum')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Time_spent_chating_H:M', 'transformation_label': 'String Indexer'}], 'feature': 'Time_spent_chating_H:M', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '0:0', 'max': '9:57', 'missing': '0', 'distinct': '127'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Time_spent_chating_H:M'}, {'feature_label': 'Time_spent_chating_H:M', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Time_spent_chating_H:M')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Friends_ID_list', 'transformation_label': 'String Indexer'}], 'feature': 'Friends_ID_list', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '103', 'mean': '15688.73', 'stddev': '11606.95', 'min': '10087', 'max': '9483,6987', 'missing': '397', 'distinct': '102'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Friends_ID_list'}, {'feature_label': 'Friends_ID_list', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Friends_ID_list')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Risk', 'transformation_label': 'String Indexer'}], 'feature': 'Risk', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '0', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '500', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Risk'}, {'feature_label': 'Risk', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Risk')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run thoseatriskofonlinesexworkHooks.ipynb
try:
	#sourcePreExecutionHook()

	onlinesexwork = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/online_sex_work.csv', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'delimiter': ',', 'is_header': 'Use Header Line', 'filename': 'online_sex_work.csv', 'results_url': 'http://ml.colaberry.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run thoseatriskofonlinesexworkHooks.ipynb
try:
	#transformationPreExecutionHook()

	classificationsofthoseatrisk = TransformationMain.run(onlinesexwork,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "User_ID", "threshold": 12441.148, "transformation_label": "Binarizer"}], "feature": "User_ID", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "12441.15", "stddev": "8613.12", "min": "1.0", "max": "48294.0", "missing": "0"}, "transformation": [{"transformation": "Binarizer", "selectedAsDefault": 1}], "updatedLabel": "User_ID"}, {"transformationsData": [{"feature_label": "Gender", "transformation_label": "String Indexer"}], "feature": "Gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "female", "max": "male", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Gender"}, {"transformationsData": [{"feature_label": "Age", "transformation_label": "String Indexer"}], "feature": "Age", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "38.4", "stddev": "10.68", "min": "20,7", "max": "86,6", "missing": "0", "distinct": "247"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Age"}, {"transformationsData": [{"feature_label": "Location", "transformation_label": "String Indexer"}], "feature": "Location", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "A", "max": "S", "missing": "0", "distinct": "15"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Location"}, {"transformationsData": [{"feature_label": "Verification", "transformation_label": "String Indexer"}], "feature": "Verification", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Administrator", "max": "Verified", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Verification"}, {"transformationsData": [{"feature_label": "Sexual_orientation", "transformation_label": "String Indexer"}], "feature": "Sexual_orientation", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "499", "mean": "", "stddev": "", "min": "Heterosexual", "max": "bisexual", "missing": "1", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Sexual_orientation"}, {"transformationsData": [{"feature_label": "Sexual_polarity", "transformation_label": "String Indexer"}], "feature": "Sexual_polarity", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "499", "mean": "", "stddev": "", "min": "Dominant", "max": "Switch", "missing": "1", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Sexual_polarity"}, {"transformationsData": [{"feature_label": "Looking_for", "transformation_label": "String Indexer"}], "feature": "Looking_for", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "495", "mean": "", "stddev": "", "min": "Men", "max": "Women", "missing": "5", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Looking_for"}, {"transformationsData": [{"feature_label": "Points_Rank", "transformation_label": "String Indexer"}], "feature": "Points_Rank", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "27.84", "stddev": "83.83", "min": "-100", "max": "a", "missing": "0", "distinct": "83"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Points_Rank"}, {"transformationsData": [{"feature_label": "Last_login", "transformation_label": "String Indexer"}], "feature": "Last_login", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "before_1003_days", "max": "before_9_days", "missing": "0", "distinct": "250"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Last_login"}, {"transformationsData": [{"feature_label": "Member_since", "transformation_label": "String Indexer"}], "feature": "Member_since", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "1.11.2009", "max": "9.9.2015", "missing": "0", "distinct": "112"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Member_since"}, {"transformationsData": [{"feature_label": "Number_of_Comments_in_public_forum", "transformation_label": "String Indexer"}], "feature": "Number_of_Comments_in_public_forum", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "20.59", "stddev": "93.3", "min": "0", "max": "96", "missing": "0", "distinct": "65"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Number_of_Comments_in_pub..."}, {"transformationsData": [{"feature_label": "Time_spent_chating_H:M", "transformation_label": "String Indexer"}], "feature": "Time_spent_chating_H:M", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "0:0", "max": "9:57", "missing": "0", "distinct": "127"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Time_spent_chating_H:M"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Number_of_advertisments_posted", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.26", "stddev": "1.04", "min": "0.0", "max": "17.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Number_of_advertisments_p..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Number_of_offline_meetings_attended", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.11", "stddev": "0.81", "min": "0.0", "max": "11.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Number_of_offline_meeting..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Profile_pictures", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.96", "stddev": "4.68", "min": "0.0", "max": "48.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Profile_pictures"}, {"transformationsData": [{"feature_label": "Friends_ID_list", "transformation_label": "String Indexer"}], "feature": "Friends_ID_list", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "103", "mean": "15688.73", "stddev": "11606.95", "min": "10087", "max": "9483,6987", "missing": "397", "distinct": "102"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Friends_ID_list"}, {"transformationsData": [{"feature_label": "Risk", "transformation_label": "String Indexer"}], "feature": "Risk", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "0", "mean": "", "stddev": "", "min": "", "max": "", "missing": "500", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Risk"}]}))

	#transformationPostExecutionHook(classificationsofthoseatrisk)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run thoseatriskofonlinesexworkHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(classificationsofthoseatrisk, ["Number_of_advertisments_posted", "Number_of_offline_meetings_attended", "Profile_pictures", "User_ID_binarizer", "Gender_stringindexer", "Age_stringindexer", "Location_stringindexer", "Verification_stringindexer", "Sexual_orientation_stringindexer", "Sexual_polarity_stringindexer", "Looking_for_stringindexer", "Points_Rank_stringindexer", "Last_login_stringindexer", "Member_since_stringindexer", "Number_of_Comments_in_public_forum_stringindexer", "Time_spent_chating_H:M_stringindexer", "Friends_ID_list_stringindexer"], "Risk_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

